# Osteisarcoma Label Cleaning Multiple Instance Learning

#### 0. Import Packages

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.utils.data as data_utils
from torchvision import datasets, transforms
from scipy.stats import binom
import cv2 as cv
import numpy as np
from skimage import morphology
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import random
import openslide
import numpy as np
import random
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import cv2 as cv
from skimage import morphology
import matplotlib.pyplot as plt
import wandb
import os
from os.path import basename
import json 
import pandas as pd
from __future__ import print_function, division
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import pdb
from torch.utils.data import Dataset, DataLoader
from glob import glob
from skimage import io, transform
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau


#### 1. Basic Slide information 

In [3]:
# Path to your WSI file
wsi_path_1 = '/cis/home/jhu104/jhu101/OTS-23-17323 - 2023-06-15 18.31.27.ndpi'
wsi_label_path_1 = "/export/io86/data/jhu101/OTS-23-17323 - 2023-06-15 18.31.27.ndpi.xml"

#Open the WSI file
slide = openslide.OpenSlide(wsi_path_1)
# print(f"Slide Level Count:{slide.level_count}")
# print(f"Slide Dimensions:{slide.dimensions}")
# print(f"Slide level dimensions:{slide.level_dimensions}")

# 1.Generate Tissue Mask

In [4]:
def binary_Aaron(img,adjust_otsu, fill_size=50, remove_size=50):
    # 16714505 1.21
    # 16714503 1.25
    #Changing the BGR Channel to Gray
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    #Setting a threhold where >threhold =255; <threhold =0. This is for denoising
    otsu_threshold, _ = cv.threshold(gray, 0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    #Apply to our downsampling WSI
    binary = gray<= (otsu_threshold*adjust_otsu)
    #Don't quite understand this part? Why should we do this.
    binary = morphology.remove_small_objects(morphology.remove_small_holes(binary, fill_size),remove_size)
    return binary

    

In [5]:
def generate_tissue_mask(slide_ob,unit,adjust_otsu):
    """
    Generate tissue mask (downsampled) for WSI
    - Input
        slide_ob: slide object
        unit: downsample scale
        adjust_otsu: adjust the OTSU threshold
    - Return
        mask tissue
    """
    #Get the width and height of WSI
    width,height = slide_ob.dimensions
    #Get the downsampling width and height of WSI
    width_downsample, height_downsample = width//unit, height//unit
    #Get the thumnail with respect to the down-sampling WSI 
    thumbnail = slide_ob.get_thumbnail((width_downsample,width_downsample))
    #Using opencv to resize the thumnial. Aim to convert the image to numpy array
    thumbnail = cv.resize(np.array(thumbnail)[:,:,:3],(width_downsample,height_downsample))
    #Use binary to denoise our image and convert it to numpy array
    mask_tissue = np.array(binary_Aaron(thumbnail, adjust_otsu),dtype=np.uint8) 
    return mask_tissue


In [6]:
#Generate label mask
def generate_label_mask(slide_ob, annotations, annotation_label_mapping, unit):
    """
    Generate label mask (downsampled) for WSI
    - Input
        slide_ob: slide object
        annotations: {'annotation_key':{'outer': [(x,y),....],'inner':[(x,y),...]}}
        'annotation_label_mapping': {'annotation_key': int}
        unit: downsample scale
    - Return
        Mask: label mask
    """
    width, height = slide_ob.dimensions
    Mask = np.zeros((int(height/unit),int(width/unit)),dtype=float)
    for annotation_key in annotations.keys():
        mask =  Image.new('1', (int(np.round(width/unit)),int(np.round(height/unit))))
        draw = ImageDraw.Draw(mask)
        for contour in annotations[annotation_key]['outer']:
            contour = [(i[0]/unit,i[1]/unit) for i in contour]
            draw.polygon(contour,fill=1,outline=0)
        for contour in annotations[annotation_key]['inner']:
            contour = [(i[0]/unit,i[1]/unit) for i in contour]
            draw.polygon(contour,fill=0,outline=0)
        mask = np.array(mask)
        Mask[mask==1] = annotation_label_mapping[annotation_key]
    return Mask

In [7]:
# adjust_otsu =1
# mask_tissue = generate_tissue_mask(slide,256,adjust_otsu)
# plt.imshow(mask_tissue)
# plt.show()

# 2. Generate Mask Label

In [8]:
def generate_label_mask(slide_ob, annotations, annotation_label_mapping, unit):
    """
    Generate label mask (downsampled) for WSI
    - Input
        slide_ob: slide object
        annotations: {'annotation_key':{'outer': [(x,y),....],'inner':[(x,y),...]}}
        'annotation_label_mapping': {'annotation_key': int}
        unit: downsample scale
    - Return
        Mask: label mask
    """
    #Get the width and height of our WSI
    width, height = slide_ob.dimensions
    #Build an empty label mask matrix for storing the labels later
    Mask = np.zeros((int(height/unit),int(width/unit)),dtype=float)
    #Find the labels for each classes: Viable, necrosis and stroma
    for annotation_key in annotations.keys():
      #Create a new image with mode 1(1-bit pixels,black and white)
        mask =  Image.new('1', (int(np.round(width/unit)),int(np.round(height/unit))))
        #Draw the labels for each class
        draw = ImageDraw.Draw(mask)
        #Get the coordinates information for each location inside one class
        for contour in annotations[annotation_key]['outer']:
          #Consider the downsampling factor
            contour = [(i[0]/unit,i[1]/unit) for i in contour]
            #Use polygon to connect the vetex and draw the regions wrt classes
            draw.polygon(contour,fill=1,outline=0)
        for contour in annotations[annotation_key]['inner']:
            contour = [(i[0]/unit,i[1]/unit) for i in contour]
            draw.polygon(contour,fill=0,outline=0)
        #Convert our created image into arrays(each mask representing the labels for each class)
        mask = np.array(mask)
        #Mapping all our class labels to one big matrix, which means the matrix will contain all the labels(In our case,1 for viable,2 for necrosis, 3 for stroma )
        Mask[mask==1] = annotation_label_mapping[annotation_key]
    return Mask


In [9]:
def read_Aaron_annotations(xml_path):
    root = ET.parse(xml_path)
    Annotations={'viable':{'outer':[], 'inner':[]},
                'necrosis':{'outer':[], 'inner':[]},
                'stroma':{'outer':[], 'inner':[]},
                'type4':{'outer':[], 'inner':[]}}
    for a in root.iter('Annotation'):
        for r in a.iter('Region'):
            Annotation = []
            for v in r.iter('Vertex'):
                Annotation.append((float(v.attrib['X']), float(v.attrib['Y'])))
            if a.attrib['LineColor'] == '16711680' :
                Annotations['viable']['outer'].append(Annotation)
            elif a.attrib['LineColor'] == '255':
                Annotations['necrosis']['outer'].append(Annotation)
            elif a.attrib['LineColor'] == '65280' or a.attrib['LineColor'] == '1376057':
                Annotations['stroma']['outer'].append(Annotation)
            elif a.attrib['LineColor'] == '65535':
                Annotations['type4']['outer'].append(Annotation)
    return Annotations

In [10]:
#set Annotation mapping as following:
annotation_label_mapping ={
    'stroma':1,
    'viable':2,
    'necrosis':3,
    'type4':4
}
#Read the annotations
Annotations=read_Aaron_annotations(wsi_label_path_1)
# Annotations2=read_Aaron_annotations(wsi_label_path_2)
#Get the label Mask
Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
# Mask2 = generate_label_mask(slide2,Annotations2,annotation_label_mapping,256)
# Mask_dic = {"s1":Mask,"s2":Mask2}
np.sum((np.array(Mask))==2)
# plt.imshow(Mask)

514

# 3. Prepare the Dataset

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.utils.data as data_utils
from torchvision import datasets, transforms
from scipy.stats import binom
import cv2 as cv
import numpy as np
from skimage import morphology
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import random

#### 3.1 Dataset: Mixslide Bags

In [12]:
# class TSbags_random_mixpatch2(data_utils.Dataset):
#     def __init__(self, slide_obs, masks_tissue, masks_label, level, patch_shape, length_bag_mean, num_bags, transform,mode='train'):
#         self.slide_obs = slide_obs
#         self.masks_tissue = masks_tissue
#         self.masks_label = masks_label
#         self.level = level
#         self.patch_shape =patch_shape
#         self.length_bag_mean = length_bag_mean
#         self.num_bags = num_bags
#         self.transform = transform
#         self.Patch_list, self.Label_patch_list = self._mix_patches()
#         self.train_bags_list,self.train_labels_list,self.val_bags_list,self.val_labels_list = self._create_bags()
#         self.mode = mode


#     def _mix_patches(self):
#         # to return: [(slide_ID,row,col)]
#         Patch_list = []
#         Label_patch_list = []
#         #Iterate though each slide
#         for slide_ID in self.slide_obs.keys():
#           #Find the tissue mask and label for each slide
#             # print("")
#             # print(slide_ID)
#             mask_tissue = self.masks_tissue[slide_ID]
#             mask_label = self.masks_label[slide_ID]
#             for label in range(len(self.num_bags)):
#               #Find all the pixels belong to tissue and correponding class
#                 ROW, COL = np.where((mask_tissue==1)&(mask_label==(label+1))) # 给出俩个条件均满足的坐标，如果mask_label！=label，则是空集，如果mask_label>3，none
#                 #print(f"Type 4: {np.sum(mask_label==1)}")

#                 #Aggregate all the pixels belong to tissue in a long list
#                 Patch_list.extend([(slide_ID,ROW[i],COL[i]) for i in range(len(ROW))])
#                 #Aggregate all the labels as well
#                 Label_patch_list.extend([label+1]*len(ROW))
#         return Patch_list, np.array(Label_patch_list)


#     def _create_bags(self):
#         train_bags_list = [] #len=total number of classes. Each list inside is the patch coordinates.
#         train_labels_list = []
#         val_bags_list = []
#         val_labels_list=[]
#       #  print(self.num_bags)
#         for label in range(len(self.num_bags)): #label=0,1,2
#             #print(label+1)
#           #Index where the Label_patch_list ==1 eg
#             Indices = np.where(self.Label_patch_list==(label+1))[0] #Indices (219602,) pxiels where the patch_list label is the same as the label
#             #print(f"{label+1}:{len(Indices)}")
#             # random.shuffle(Indices)
#             random.seed(42)
#             shuffled_indices = Indices[:]
#             split_idx = int(0.7*len(Indices))
#             # Slices dataset into train and validation
#             train_indices = shuffled_indices[:split_idx]
#             # print(len(train_indices))
#             val_indices = shuffled_indices[split_idx:]
#             #Find the coordinates of each class bags respectively and append them
#             #print(f"val_indices: {len(train_indices)}")

#             #Train bags
#             for bag_idx in range(self.num_bags[label]):
#                 length_bag = binom.rvs (n=int(self.length_bag_mean*2), p=0.5) #randomly sampling bag length mean
#                 indices = random.sample(train_indices.tolist(),length_bag) #Indices (219602,) 取samples等于bag length 
#                 train_bags_list.append(indices)
#                 train_labels_list.append(label)

#             #Validation bags
#             for bag_idx in range(int(0.5*self.num_bags[label])):
#                 length_bag = binom.rvs (n=int(self.length_bag_mean*2), p=0.5) #randomly sampling bag length mean
#                 indices = random.sample(val_indices.tolist(),length_bag) #Indices (219602,) 取samples等于bag length 
#                 val_bags_list.append(indices)
#                 val_labels_list.append(label)
#         return train_bags_list,train_labels_list,val_bags_list,val_labels_list

#         # return bags_list, labels_list
    
#     def _pack_one_bag(self,indices):
#         Bag = []
#         for index in indices:
#             slide_ID, row_unit, col_unit = self.Patch_list[index]
#             factor = self.slide_obs[slide_ID].level_downsamples[self.level]
#             unit = int(self.slide_obs[slide_ID].dimensions[0]/self.masks_tissue[slide_ID].shape[1])
#             upperLeft_x = int(col_unit * unit + unit/2 - self.patch_shape/2*factor) # Pick the upperleft x coordinate
#             upperLeft_y = int(row_unit * unit + unit/2 - self.patch_shape/2*factor) # Pick the upperleft y coordinate
#             #Find patch using read_region in open slide
#             patch = self.slide_obs[slide_ID].read_region((upperLeft_x, upperLeft_y),self.level,(self.patch_shape,self.patch_shape))
#             patch = Image.fromarray(np.array(patch)[:,:,:3])
#             if self.transform is not None:
#                 patch = self.transform(patch)
#             Bag.append(patch) # Stack bags into one numpy array
#         #Stack patches into bags
#         Bag = np.stack(Bag,axis=0)
#         return Bag
#     def __len__(self):
#         if self.mode == "train":
#             return len(self.train_bags_list)
#         else:
#             return len(self.val_bags_list)
    
#     def __getitem__(self, index):
#         if self.mode == "train":
#             indices = self.train_bags_list[index]
#             bag = self._pack_one_bag(indices)
#             label = self.train_labels_list[index]
#             return bag, label
#         else: 
#             indices = self.val_bags_list[index]
#             bag = self._pack_one_bag(indices)
#             label = self.val_labels_list[index]
#             return bag, label

In [13]:
class TSbags_random_mixpatch(data_utils.Dataset):
    def __init__(self, slide_obs, masks_tissue, masks_label, level, patch_shape, length_bag_mean, num_bags, transform):
        self.slide_obs = slide_obs
        self.masks_tissue = masks_tissue
        self.masks_label = masks_label
        self.level = level
        self.patch_shape =patch_shape
        self.length_bag_mean = length_bag_mean
        self.num_bags = num_bags
        self.transform = transform
        self.Patch_list, self.Label_patch_list = self._mix_patches()
        self.bags_list, self.labels_list = self._create_bags()  
        
    def _mix_patches(self):
        # to return: [(slide_ID,row,col)]
        Patch_list = []
        Label_patch_list = []
        for slide_ID in self.slide_obs.keys():
            mask_tissue = self.masks_tissue[slide_ID]
            mask_label = self.masks_label[slide_ID]
            for label in range(len(self.num_bags)):
                ROW, COL = np.where((mask_tissue==1)&(mask_label==(label+1)))
                Patch_list.extend([(slide_ID,ROW[i],COL[i]) for i in range(len(ROW))])
                Label_patch_list.extend([(slide_ID, label) for _ in ROW])
        return Patch_list, Label_patch_list
            
    def _create_bags(self):            
        bags_list = []
        labels_list = []
        for slide_ID in self.slide_obs.keys():
            # Retrieve both the label and the global index for patches from the current slide
            patches_for_slide = [(index, label) for index, (s_id, label) in enumerate(self.Label_patch_list) if s_id == slide_ID]
            path_for_oneslide = np.array([label for index, label in patches_for_slide])
            global_indices = [index for index, label in patches_for_slide]

            for label in range(len(self.num_bags)):
                # Find local indices of the patches with the specific label
                local_label_indices = np.where(path_for_oneslide == label)[0]
                # Map local indices to global indices
                label_indices = [global_indices[i] for i in local_label_indices]

                for bag_idx in range(self.num_bags[label]):
                    if len(label_indices) <= self.length_bag_mean*2+self.length_bag_mean/2:
                        pass
                    else:
                        length_bag = binom.rvs(n=int(self.length_bag_mean * 2), p=0.5)
                        # Sample from the global indices
                        selected_indices = random.sample(label_indices, length_bag)
                        bags_list.append(selected_indices)
                        labels_list.append(label)

        return bags_list, labels_list

    def _pack_one_bag(self,indices):
        Bag = []
        for index in indices:
            slide_ID, row_unit, col_unit = self.Patch_list[index]
            factor = self.slide_obs[slide_ID].level_downsamples[self.level]
            unit = int(self.slide_obs[slide_ID].dimensions[0]/self.masks_tissue[slide_ID].shape[1])
            upperLeft_x = int(col_unit * unit + unit/2 - self.patch_shape/2*factor)
            upperLeft_y = int(row_unit * unit + unit/2 - self.patch_shape/2*factor)
            patch = self.slide_obs[slide_ID].read_region((upperLeft_x, upperLeft_y),self.level,(self.patch_shape,self.patch_shape))
            patch = Image.fromarray(np.array(patch)[:,:,:3])
            if self.transform is not None:
                patch = self.transform(patch)
            Bag.append(patch)
        Bag = np.stack(Bag,axis=0)
        return Bag  
    def __len__(self):
        return len(self.bags_list)  
    def __getitem__(self, index):
        indices = self.bags_list[index]
        bag = self._pack_one_bag(indices)
        label = self.labels_list[index]
        return bag, label

In [14]:
# #Mix patches
# def create_dataset_mixpatch2(slides, tissue_masks, label_masks, num_bags, level, patch_shape,length_bag_mean = 10,mode="train"):
#     """
#     Generate data loaders
#     - Input
#         slides: dictionary {'slide_ID':slide_ob}
#         tissue_masks: dictionary {'slide_ID':array}
#         label_masks: dictionary {'slide_ID':array}
#         num_bags:list
#         level:int
#         patch_shape:int
#     - Return
#         Dataset
#     """
#     # Training loaders
#     Dataset = TSbags_random_mixpatch(slide_obs = slides,
#                                             masks_tissue = tissue_masks, 
#                                             masks_label = label_masks, 
#                                             level = level, 
#                                             patch_shape = patch_shape, 
#                                             length_bag_mean = length_bag_mean, 
#                                             num_bags = num_bags, 
#                                           transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor(),
#                                           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]),
#                                           mode=mode)      
#     return Dataset

In [15]:
#Mix patches
def create_dataset_mixpatch(slides, tissue_masks, label_masks, num_bags, level, patch_shape,length_bag_mean = 10):
    """
    Generate data loaders
    - Input
        slides: dictionary {'slide_ID':slide_ob}
        tissue_masks: dictionary {'slide_ID':array}
        label_masks: dictionary {'slide_ID':array}
        num_bags:list
        level:int
        patch_shape:int
    - Return
        Dataset
    """
    # Training loaders
    Dataset = TSbags_random_mixpatch(slide_obs = slides,
                                            masks_tissue = tissue_masks, 
                                            masks_label = label_masks, 
                                            level = level, 
                                            patch_shape = patch_shape, 
                                            length_bag_mean = length_bag_mean, 
                                            num_bags = num_bags, 
                                          transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))      
    return Dataset

# 4. Define the Model

#### 4.1 LC-MIL model architecture

In [16]:
class Attention_modern_multi(nn.Module):
    def __init__(self,cnn,focal_loss=False):
        super(Attention_modern_multi,self).__init__()
        #Attention Pooling input dimension
        self.L = 1000
        self.D = 64
        self.K = 1
        self.focal_loss = focal_loss
        #Feature_extracter vgg16 with first two child weights freezed
        self.feature_extractor = cnn
        #Attention Pooling
        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K))
        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 4)
        )
    def forward(self,x):
        x = x .squeeze(0)

        H = self.feature_extractor(x)

        A = self.attention(H)
        A = torch.transpose(A,1,0)
        #Rescale attention weights between 0 and 1
        A = F.softmax(A,dim=1)
        M = torch.mm(A,H)
        Y_prob = self.classifier(M)
        #Convert Raw logit to probability
        Y_prob = F.softmax(Y_prob,dim=1)

        return Y_prob, A
    def calculate_classification_error(self, X, Y):
        Y_prob,_ = self.forward(X)
        #Choose the class with the max probability
        Y_hat = torch.argmax(Y_prob[0])
        #Calculate the classification accuracy
        error = 1. - Y_hat.eq(Y).cpu().float().mean().data.item()

        return error, Y_hat, Y_prob
    def calculate_objective(self, X, Y):
        Y_prob, A = self.forward(X) # 这里的ylabel应该是0，1，2，3，所以dataloader里面的label应该减去1
        #print(f"Y_prob:{Y_prob}")
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        if not self.focal_loss:
            loss = nn.CrossEntropyLoss()
            #print(f"focal_loss; label:{Y}")
            neg_log_likelihood = loss(Y_prob, Y)
        else:
            #print(f"label:{Y}")
            Y_prob_target = Y_prob[0,Y.cpu().data]
            #print(f"else; Y_prob_target:{Y_prob_target}")
            if Y_prob_target.cpu().data.numpy()[0]<0.2:
                gamma = 5
            else:
                gamma = 3
            neg_log_likelihood =-1. *(1-Y_prob_target)**gamma* torch.log(Y_prob_target)
        return neg_log_likelihood, A

In [17]:
def load_vgg16():
    vgg = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
    num_layer = 0
    #Fine tune the fully connect layer
    for child in vgg.children():
        print(child)
        num_layer+=1
        if num_layer < 3:
            print(child)
            for param in child.parameters():
                param.requires_grad = False #Freeze covolutional and pooling layer
    return vgg

In [18]:
def load_vgg16_tune():
    vgg = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
    for param in vgg.features.parameters():
        param.requires_grad = False
    
    # for layer_num in range(10,31):
    #     for param in vgg.features[layer_num].parameters():
    #         param.requires_grad = True
    
    # for layer_num,child in enumerate(vgg.features.children()):
    #     for param in child.parameters():
             # print(f'Layer {layer_num}, requires_grad: {param.requires_grad}')
    for layer_num in range(21,31):  # Unfreeze the last few layers
        for param in vgg.features[layer_num].parameters():
            param.requires_grad = True

    # Replace the classifier with a smaller one or add dropout and batch normalization
    # vgg.classifier = nn.Sequential(
    #     nn.Linear(25088, 4096),
    #     nn.ReLU(),
    #     nn.Dropout(0.65),  # Retain dropout at 0.5 or adjust
    #     nn.Linear(4096, 1024),  # Reduce the size to prevent overfitting
    #     nn.ReLU(),
    #     nn.Dropout(0.65),  # Increase dropout if needed
    #     nn.Linear(1024, vgg.classifier[-1].out_features))

    return vgg

In [19]:
def load_resnet():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)

    for param in model.parameters():
        param.requires_grad = False
    

    indices = [-3, -1]
    for i in indices:  
        for param in model.layer3[i].parameters():
            param.requires_grad = True

    for param in model.fc.parameters():
        param.requires_grad = True

    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # for layer_num,child in enumerate(model.children()):
    #     for param in child.parameters():
    #         print(f'Layer {layer_num}, requires_grad: {param.requires_grad}')

    return model
    

In [20]:
def load_resnet_tune():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)

    for param in model.parameters():
        param.requires_grad = False

    for param in model.fc.parameters():
        param.requires_grad = True

    for param in model.layer4.parameters():
        param.requires_grad = True
        
    # for layer_num,child in enumerate(model.children()):
    #     for param in child.parameters():
    #         print(f'Layer {layer_num}, requires_grad: {param.requires_grad}')

    return model

In [21]:
# transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor()])
# data = TSbags_random_mixpatch(slides_train,tissue_masks_train,label_masks_train, level, patch_shape,10,train_bag_len,transform)
# n_val =len(data)
# split_train =100
# indices_val = random.sample(list(range(n_val)),k=100)
# Sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
# Dataloader_val = data_utils.DataLoader(data, sampler = Sampler_val, batch_size = 1, shuffle = False)

In [22]:
# # device = torch.device("cuda:6")
# # torch.cuda.set_device(device)
# model = Attention_modern_multi(load_vgg16(),True)
# # model = model.to(device)
# for batch_idx, (data, label) in enumerate(Dataloader_val):
#     bag_label = label
#     # data, bag_label = data.to(device), bag_label.to(device)
#     data, bag_label = Variable(data), Variable(bag_label)
#     loss, _ = model.calculate_objective(data, bag_label)
#     print(loss)

#### 4.2 Training Functions

In [23]:
def train(model, optimizer, Dataloader_train):
    model.train()
    train_loss = 0.
    train_error = 0.
    optimizer.zero_grad()
    for batch_idx, (data, label) in enumerate(Dataloader_train):
        bag_label = label
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, _ = model.calculate_objective(data, bag_label)
        error, _, _ = model.calculate_classification_error(data, bag_label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.data.cpu().numpy()[0]
        train_error += error
        del data
        del bag_label
    train_loss /= len(Dataloader_train)
    train_error /= len(Dataloader_train)
    return model, train_loss, 1-train_error

In [24]:
def val(model, Dataloader_val):
    model.eval()
    val_loss = 0.
    val_error = 0.      
    for batch_idx, (data, label) in enumerate(Dataloader_val):
        bag_label = label
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data,requires_grad=False), Variable(bag_label,requires_grad=False)
        loss, _ = model.calculate_objective(data, bag_label)
        error, _, _ = model.calculate_classification_error(data, bag_label)
        val_loss += loss.data.cpu().numpy()[0]
        val_error += error
        del data
        del bag_label
    val_loss /= len(Dataloader_val)
    val_error /= len(Dataloader_val)
    return val_loss, 1-val_error

In [25]:
def test(model, Dataloader_test):
    test_error = 0. 
    pred_labels = []   
    true_labels = [] 
    for batch_idx, (data, label) in enumerate(Dataloader_test):
        bag_label = label
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data,requires_grad=False), Variable(bag_label,requires_grad=False)
        error, pred, prob = model.calculate_classification_error(data, bag_label)
        # print(prob.cpu().data.numpy()[0])
        pred_labels.append(pred.data.cpu().item())
        true_labels.append(bag_label.data.cpu().item())
        test_error += error
        if batch_idx%299 ==0:
            print(f"{batch_idx}th test accuracy:{1-(test_error/(batch_idx+1)):.3f}")
        del data
        del bag_label
    test_error /= len(Dataloader_test)
    return pred_labels, true_labels,1-test_error

In [26]:
def Train(model, Dataset_train, optimizer, scheduler, fold_number,validation=False, Dataset_val=None,patience=6):
    wandb.watch(model, log="all", log_freq=10)
    
    n_train = len(Dataset_train)
    split_train = 200
    indices_train = random.sample(list(range(n_train)), k=n_train)
    Train_loss = []
    Train_accuracy = []
    Val_loss = []
    Val_accuracy = []

    best_val_loss = float('inf')
    epochs_no_improve = 0

    if validation:
        n_val = len(Dataset_val)
        indices_val = random.sample(list(range(n_val)), k=1000)

    for i in range(n_train // split_train):
        print(f"epoch: {i}")
        Sampler_train = torch.utils.data.sampler.SubsetRandomSampler(indices_train[i*split_train:(i+1)*split_train])
        Dataloader_train = data_utils.DataLoader(Dataset_train, sampler=Sampler_train, batch_size=1, shuffle=False)
        model.cuda()
        model, train_loss, train_accuracy = train(model, optimizer, Dataloader_train)
        Train_loss.append(train_loss)
        Train_accuracy.append(train_accuracy)
        scheduler.step()

        current_lr = optimizer.param_groups[0]['lr']
        wandb.log({"train_loss": train_loss, "train_accuracy": train_accuracy, "learning_rate": current_lr})
        print("epoch={}/{}, train loss = {:.3f}, train_accuracy = {:.3f}".format(i, n_train//split_train, train_loss, train_accuracy))

        if validation:
            Sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
            Dataloader_val = data_utils.DataLoader(Dataset_val, sampler=Sampler_val, batch_size=1, shuffle=False)
            val_loss, val_accuracy = val(model, Dataloader_val)
            Val_loss.append(val_loss)
            Val_accuracy.append(val_accuracy)
            wandb.log({"val_loss": val_loss, "val_accuracy": val_accuracy})
            print("epoch={}/{}, val loss = {:.3f}, val_accuracy = {:.3f}".format(i, n_train//split_train, val_loss, val_accuracy))

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
                torch.save(model.state_dict(), f"/cis/home/jhu104/osteosarcoma/cross_validation_fold1/best_model_{fold_number}.pth")
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print("Early stopping triggered")
                    break

        if i % 2 == 0:
            torch.save(model.state_dict(), f"/cis/home/jhu104/osteosarcoma/cross_validation_fold1/ml_{i}_epoch{fold_number}.pth")

    torch.save(model.state_dict(), f"/cis/home/jhu104/osteosarcoma/cross_validation_fold1/ml_final{fold_number}.pth")
    return model, Train_loss, Train_accuracy, Val_loss, Val_accuracy


In [27]:
class TSbags_random_oneslide(data_utils.Dataset):
    def __init__(self, slide_ob, mask_tissue, mask_label, level, patch_shape, length_bag_mean, num_bags, transform):
        self.slide_ob = slide_ob
        self.mask_tissue = mask_tissue
        self.mask_label = mask_label
        self.level = level
        self.patch_shape =patch_shape
        self.length_bag_mean = length_bag_mean
        self.num_bags = num_bags
        self.transform = transform
        self.unit = int(self.slide_ob.dimensions[0]/mask_tissue.shape[1])
        self.bags_list, self.labels_list = self._create_bags()  
    def _create_bags(self):            
        bags_list = []
        labels_list = []
        for label in range(len(self.num_bags)):         
            ROW, COL = np.where((self.mask_tissue==1)&(self.mask_label==label+1))
            for bag_idx in range(self.num_bags[label]):
                if len(ROW) <=self.length_bag_mean:
                    pass
                else: 
                    length_bag = binom.rvs (n=int(self.length_bag_mean*2), p=0.5)
                    indices = np.random.randint(0,len(ROW),length_bag)
                    bags_list.append((ROW[indices], COL[indices]))
                    labels_list.append(label)
        return bags_list, labels_list
    def _pack_one_bag(self,row_list, col_list):
        Bag = []
        for i in range(len(row_list)):
            row_unit, col_unit = row_list[i], col_list[i]
            factor = self.slide_ob.level_downsamples[self.level]
            upperLeft_x = int(col_unit * self.unit + self.unit/2 - self.patch_shape/2*factor)
            upperLeft_y = int(row_unit * self.unit + self.unit/2 - self.patch_shape/2*factor)
            patch = self.slide_ob.read_region((upperLeft_x, upperLeft_y),self.level,(self.patch_shape,self.patch_shape))
            patch = Image.fromarray(np.array(patch)[:,:,:3])
            if self.transform is not None:
                patch = self.transform(patch)
            Bag.append(patch)
        Bag = np.stack(Bag,axis=0)
        return Bag  
    def __len__(self):
        return len(self.bags_list)  
    def __getitem__(self, index):
        row_list, col_list = self.bags_list[index]
        bag = self._pack_one_bag(row_list, col_list)
        label = self.labels_list[index]
        return bag, label

In [28]:
def create_dataset_mixbag(slides, tissue_masks, label_masks, num_bags, level, patch_shape,length_bag_mean = 10):
    """
    Generate data loaders
    - Input
        slides: dictionary {'slide_ID':slide_ob}
        tissue_masks: dictionary {'slide_ID':array}
        label_masks: dictionary {'slide_ID':array}
        num_bags:dict{'slide_ID':list}
        level:int
        patch:int
    - Return
        Dataset
    """
    # Training loaders
    dataset = TSbags_random_oneslide(slide_ob = slides,
                                        mask_tissue = tissue_masks, 
                                        mask_label = label_masks, 
                                        level = level, 
                                        patch_shape = patch_shape, 
                                        length_bag_mean = length_bag_mean, 
                                        num_bags = num_bags, 
                                        transform = transforms.Compose([transforms.Resize(224),transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))      
    return dataset

# 5A. Training Using Multiple slides

#### 5.0 OSTU setting

In [32]:
#step0
#Read OSTU dictoinary
with open("/cis/home/jhu104/osteosarcoma/OSTU/ostu.dic", 'r') as j:
    adjust_otsu_all1 = json.loads(j.read())

#step1
#Get basename dictionray
data_dic = {}
for key,value in adjust_otsu_all1.items():
    basename = os.path.basename(key)
    data_dic[basename] = value

fold_index = [0,1,2]
accuracy = {}
for fold in range(len(fold_index)):
    print(f"!!!!!!!!!!!!Fold:{fold_index[fold]}")
    #step2
    #Read training file from zhenzhen
    split_df = pd.read_csv(f"/cis/home/jhu104/osteosarcoma/OSTU/osteosarcoma_experiment_setting{fold_index[fold]}.csv")


    #step3
    #change the dictionary to df
    ostu_df = pd.DataFrame(data_dic.items())
    ostu_df.rename(columns={0: "image_name", 1: "OSTU"}, inplace=True)


    #step4
    #merge the table
    split_df=split_df.merge(ostu_df,on = ["image_name"])
    split_df=split_df.sort_values(by="dataset")



    #Get all the label
    label_list = []
    directory = '/export/io86/data/jhu101'
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            label_list.append(filepath)


    #Find matched label
    def find_matched_label(image,label):
        label_match = []
        for i in range(len(image)):
            appendix = os.path.basename(image[i])
            appendix = appendix+".xml"
            for j in range(len(label)):
                if appendix == os.path.basename(label_list[j]):
                    label_match.append(label_list[j])
        return label_match


    #Find all image path
    random.seed(42)
    directory = '/cis/home/jhu104/jhu101'
    datalist=[]
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            datalist.append(filepath)
    #randomly shuffle the dataset
    random.shuffle(datalist)


    #Get train,test and validation data path
    def get_data(directory):
        base = '/cis/home/jhu104/jhu101'
        datatrain=[]
        for i in range(len(directory)):
            filepath = os.path.join(base, directory[i])
            datatrain.append(filepath)
        return datatrain


    #Split into training,validation and test
    #train
    train_data = get_data(split_df[split_df["dataset"] =="training"]["image_name"].to_list())
    train_label = find_matched_label(train_data,label_list)

    #val
    val_data = get_data(split_df[split_df["dataset"] =="validation"]["image_name"].to_list())
    val_label = find_matched_label(val_data,label_list)

    #test
    test_data = get_data(split_df[split_df["dataset"] =="test"]["image_name"].to_list())
    test_label = find_matched_label(test_data,label_list)

    #Find settings for slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 10)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/2 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic

    #Get the bag length
    Settings = slide_setting(train_data,train_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides

    #Get the train setting

    adjust_otsu_train = split_df[split_df["dataset"] =="training"]["OSTU"].to_list()
    Settings =insert_otsu(Settings,adjust_otsu_train)

    #Load all training WSIs
    unit = 256
    level = 2 # 10x magnification
    patch_shape = 256
    annotation_label_mapping ={
        'stroma':1,
        'viable':2,
        'necrosis':3,
        'type4':4
    }
    slides_train = {}
    num_bags_train = {}
    annotations_train = {}
    tissue_masks_train = {}
    label_masks_train = {}
    for i, slide_ID in enumerate(train_data):
        slides_train[slide_ID] = openslide.OpenSlide(slide_ID)
        annotations_train[slide_ID] = read_Aaron_annotations(train_label[i])
        num_bags_train[slide_ID] = [Settings[slide_ID]['stroma'],Settings[slide_ID]['viable'],Settings[slide_ID]['necrosis'],Settings[slide_ID]['type4']]    
        tissue_masks_train[slide_ID] = generate_tissue_mask(slides_train[slide_ID],unit=unit,adjust_otsu=Settings[slide_ID]['adjust_otsu'])
        label_masks_train[slide_ID] = generate_label_mask(slides_train[slide_ID], annotations_train[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length
    train_bag_len =[0,0,0,0]
    for idx,key in num_bags_train.items():
        train_bag_len[0] += key[0] 
        train_bag_len[1] += key[1] 
        train_bag_len[2] += key[2] 
        train_bag_len[3] += key[3]

    #Find settings for validation slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 20)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/8 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic


    #Get the bag length
    Settings_val = slide_setting(val_data,val_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides


    adjust_otsu_val= split_df[split_df["dataset"] =="validation"]["OSTU"].to_list()
    Settings_val =insert_otsu(Settings_val,adjust_otsu_val)   


    #Load validation WSI
    if len(val_data)>0:
        slides_val = {}
        num_bags_val = {}
        annotations_val= {}
        tissue_masks_val = {}
        label_masks_val = {}
        for i, slide_ID in enumerate(val_data):
            slides_val[slide_ID] = openslide.OpenSlide(slide_ID)
            annotations_val[slide_ID] = read_Aaron_annotations(val_label[i])
            num_bags_val[slide_ID] = [Settings_val[slide_ID]['stroma'],Settings_val[slide_ID]['viable'],Settings_val[slide_ID]['necrosis'],Settings_val[slide_ID]['type4']]    
            tissue_masks_val[slide_ID] = generate_tissue_mask(slides_val[slide_ID],unit=unit,adjust_otsu=Settings_val[slide_ID]['adjust_otsu'])
            label_masks_val[slide_ID] = generate_label_mask(slides_val[slide_ID], annotations_val[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length

    val_bag_len =[0,0,0,0]
    for idx,key in num_bags_val.items():
        val_bag_len[0] += key[0] 
        val_bag_len[1] += key[1]
        val_bag_len[2] += key[2] 
        val_bag_len[3] += key[3] 

    #Find settings for test slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 20)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/8 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic


    #Get the bag length
    Settings_test = slide_setting(test_data,test_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides


    adjust_otsu_test= split_df[split_df["dataset"] =="test"]["OSTU"].to_list()
    Settings_test =insert_otsu(Settings_test,adjust_otsu_test)

    #Load test WSI
    if len(test_data)>0:
        slides_test= {}
        num_bags_test = {}
        annotations_test= {}
        tissue_masks_test = {}
        label_masks_test = {}
        for i, slide_ID in enumerate(test_data):
            slides_test[slide_ID] = openslide.OpenSlide(slide_ID)
            annotations_test[slide_ID] = read_Aaron_annotations(test_label[i])
            num_bags_test[slide_ID] = [Settings_test[slide_ID]['stroma'],Settings_test[slide_ID]['viable'],Settings_test[slide_ID]['necrosis'],Settings_test[slide_ID]['type4']]    
            tissue_masks_test[slide_ID] = generate_tissue_mask(slides_test[slide_ID],unit=unit,adjust_otsu=Settings_test[slide_ID]['adjust_otsu'])
            label_masks_test[slide_ID] = generate_label_mask(slides_test[slide_ID], annotations_test[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length
    test_bag_len =[0,0,0,0]
    for idx,key in num_bags_test.items():
        test_bag_len[0] += key[0] 
        test_bag_len[1] += key[1] 
        test_bag_len[2] += key[2]  
        test_bag_len[3] += key[3] 

    wandb.init(
        project= "vgg16",
        name = "vgg -scratch",
        config={
        "learning_rate": 1e-5,
        "architecture": "vgg16_lower_lr"
        }
    )
    random.seed(20)
    np.random.seed(20)
    torch.manual_seed(20)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(20)
    config = wandb.config
    device = torch.device("cuda:1")
    torch.cuda.set_device(device)
    model = Attention_modern_multi(load_vgg16_tune(),True)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(),lr=1e-6, betas=(0.9, 0.999), weight_decay =10e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.95)
    train_bag_len = [600,600,600,600]
    val_bag_len = [50,50,50,50] 
    Dataset_train = create_dataset_mixpatch(slides_train,
                                    tissue_masks_train, label_masks_train, 
                                    train_bag_len, level, patch_shape,length_bag_mean = 10)
    if len(slides_val)>0:
        Dataset_val = create_dataset_mixpatch(slides_val,
                                    tissue_masks_val, label_masks_val, 
                                    val_bag_len, level, patch_shape,length_bag_mean = 10)
        model, Train_loss, Train_accuracy, Val_loss, Val_accuracy = Train(model, Dataset_train, optimizer, scheduler, fold_index[fold],validation=True, Dataset_val = Dataset_val)
        
    else:
        model, Train_loss, Train_accuracy, Val_loss, Val_accuracy = Train(model, Dataset_train, optimizer, scheduler,fold_index[fold], validation=False, Dataset_val = None)

    #step0
    # load model
    #Find patient ID for each slide
    td   = split_df[split_df["dataset"]=="test"]
    test_data_dic = {}
    for index,row in td.iterrows():
        image_name = "/cis/home/jhu104/jhu101/"+ row["image_name"]
        patient_id = row["patient id"]
        test_data_dic[image_name] = patient_id
    # f = open("/cis/home/jhu104/osteosarcoma/validation and test result/test_data_dic.json")
    # test_data_dic = json.load(f)
    path = f"/cis/home/jhu104/osteosarcoma/cross_validation_fold1/best_model_{fold_index[fold]}.pth"
    # device = torch.device("cuda:0")
    model = Attention_modern_multi(load_vgg16_tune())
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    set_bag_length =50
    test_bag_len_slide = [set_bag_length,set_bag_length,set_bag_length,set_bag_length]


        # 'stroma':1,
        # 'viable':2,
        # 'necrosis':3,
        # 'type4':4


    #step1
    #Prediction accuracy    
    def prediction_accuracy(pred,true,test_bag_len_slide):
        listname = ['stroma', 'viable', 'necrosis', 'cartilage']
        label_type = int(len(pred)/test_bag_len_slide[0])
        label_name = np.unique(true)
        #Get classification result
        for i in range(label_type):
            for j in range(len(listname)):
                class_len = int(len(pred)/label_type)
                result = np.sum(np.array(pred[i*class_len:(i+1)*class_len]) ==j)/class_len
                print(f"{listname[label_name[i]]}: classification result for {listname[j]}: {result*100:.2f}%")



    #step2
    #Prediction patient accuracy
    def prediction_patient_accuracy(pred,true,test_bag_len_slide):
        listname = ['stroma', 'viable', 'necrosis', 'cartilage']
        label_type = int(len(pred)/test_bag_len_slide[0])
        label_name = np.unique(true)
        stroma_acc = []
        viable_acc= []
        necrosis_acc = []
        cartilage_acc =[]
        #Get classification result
        for i,type in enumerate(label_name):
            # print(f"prediction type:{pred}")
            class_len = int(len(pred)/label_type)
            result = (np.sum(np.array(pred[i*class_len:(i+1)*class_len]) ==type)/class_len)
            if type == 0:
                stroma_acc.append(result)
            elif type ==1:
                viable_acc.append(result)
            elif type ==2:
                necrosis_acc.append(result)
            elif type ==3:
                cartilage_acc.append(result)
        return   stroma_acc,viable_acc,necrosis_acc,cartilage_acc




    #step3
    #Compute test accuracy
    # dict(itertools.islice(test_data_dic.items(), 2,16))
    # filtered_dict = {key: value for key, value in test_data_dic.items() if value == 33}
    patient_list = td["patient id"].unique().tolist()
    patient_metrics = {patient_id: {'stroma': [], 'viable': [], 'necrosis': [], 'cartilage': []} for patient_id in patient_list}
    test_accuracy_list = {} #####Slide level accuracy

    for keys,patient_id in test_data_dic.items():
        Dataset_test = create_dataset_mixbag(slides_test[keys],
                                        tissue_masks_test[keys], label_masks_test[keys], 
                                        test_bag_len_slide, level, patch_shape,length_bag_mean = 10)

        print(f"{patient_id}th patient: {keys}")
        #Test accuracy
        Dataloader_test = data_utils.DataLoader(Dataset_test, batch_size = 1, shuffle = False)
        pred_labels_test, true_labels,test_accuracy = test(model, Dataloader_test)
        basename = os.path.basename(keys)
        test_accuracy_list[basename]=round(test_accuracy,3)
        
        #Print slide accuracy
        print("test_accuracy = {:.3f}".format(test_accuracy))
        print("Classification Results")
        prediction_accuracy(pred_labels_test,true_labels,test_bag_len_slide)
        stroma_acc,viable_acc,necrosis_acc,cartilage_acc=prediction_patient_accuracy(pred_labels_test,true_labels,test_bag_len_slide)
        
    # Append the results for this key to the patient's metrics
        if stroma_acc:  
            patient_metrics[patient_id]['stroma'].append(stroma_acc[0])
        if viable_acc:  # 
            patient_metrics[patient_id]['viable'].append(viable_acc[0])
        if necrosis_acc: 
            patient_metrics[patient_id]['necrosis'].append(necrosis_acc[0])
        if cartilage_acc:  #
            patient_metrics[patient_id]['cartilage'].append(cartilage_acc[0])
        print("")



    #Step4
    #Compute patient level accuracy
    print("Patient Level Accuracy")
    for patient_id, metrics in patient_metrics.items():
        print("")
        for metric ,values in metrics.items():
            print(f"{patient_id}th: {metric}%: {np.mean(np.array(values))*100:.2f}(std:{np.std(np.array(values))*100:.2f})")

    #step5
    #Mean Acc and Std
    print("")
    print(f"Mean Accuracy:{np.mean(np.array(list(test_accuracy_list.values())))*100:.2f}(std:{np.std(np.array(list(test_accuracy_list.values())))*100:.2f})")
    with open("/cis/home/jhu104/osteosarcoma/validation and test result/slide_accuracy", "w") as f:
        # Use json.dump to write the dictionary to the file with indentation
        json.dump(test_accuracy_list, f, indent=4)  

    test_accuracy_list_updated={key:round(value*100,3) for key, value in test_accuracy_list.items()}
    # save model
    accuracy[fold_index[fold]] = test_accuracy_list_updated
    test_accuracy_list_updated
    with open("/cis/home/jhu104/osteosarcoma/validation and test result/test_accuracy_list_updated_first.json","w") as f:
        json.dump(accuracy,f,indent=4)

!!!!!!!!!!!!Fold:0


learning_rate,█▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁▁
train_accuracy,▁▃▅▆▇▆▇▇██▇██▇█████
train_loss,█▅▄▃▂▃▂▂▁▁▂▂▂▂▁▁▁▁▁
val_accuracy,▁▃▄▅▆▆▇▇▇▆▆█▇▇█████
val_loss,█▅▄▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▂
learning_rate,0.0
train_accuracy,0.91
train_loss,0.05112
val_accuracy,0.785
val_loss,0.17073


Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch: 0
epoch=0/420, train loss = 0.509, train_accuracy = 0.400
epoch=0/420, val loss = 0.359, val_accuracy = 0.481
epoch: 1
epoch=1/420, train loss = 0.235, train_accuracy = 0.645
epoch=1/420, val loss = 0.301, val_accuracy = 0.518
epoch: 2
epoch=2/420, train loss = 0.155, train_accuracy = 0.750
epoch=2/420, val loss = 0.254, val_accuracy = 0.636
epoch: 3
epoch=3/420, train loss = 0.140, train_accuracy = 0.835
epoch=3/420, val loss = 0.252, val_accuracy = 0.630
epoch: 4
epoch=4/420, train loss = 0.153, train_accuracy = 0.790
epoch=4/420, val loss = 0.192, val_accuracy = 0.723
epoch: 5
epoch=5/420, train loss = 0.112, train_accuracy = 0.840
epoch=5/420, val loss = 0.212, val_accuracy = 0.688
epoch: 6
epoch=6/420, train loss = 0.122, train_accuracy = 0.820
epoch=6/420, val loss = 0.181, val_accuracy = 0.742
epoch: 7
epoch=7/420, train loss = 0.076, train_accuracy = 0.860
epoch=7/420, val loss = 0.180, val_accuracy = 0.749
epoch: 8
epoch=8/420, train loss = 0.099, train_accuracy = 0.835

Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0


...

learning_rate,██▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
train_accuracy,▁▄▅▆▆▆▆▇▆▆▇▇▆▇█▇▇▇▇█▇▇█▇█▇▇█████
train_loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▄▄▆▆▇▇▇▆▇▆▇▇▇█▇▇▇█████▇█▇▇███▇
val_loss,█▆▅▅▃▄▃▃▂▃▂▂▁▂▂▁▂▁▁▁▁▁▁▁▂▁▁▂▁▁▁▂
learning_rate,0.0
train_accuracy,0.94
train_loss,0.03556
val_accuracy,0.764
val_loss,0.15352


Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch: 0
epoch=0/393, train loss = 0.512, train_accuracy = 0.490
epoch=0/393, val loss = 0.323, val_accuracy = 0.587
epoch: 1
epoch=1/393, train loss = 0.252, train_accuracy = 0.620
epoch=1/393, val loss = 0.275, val_accuracy = 0.641
epoch: 2
epoch=2/393, train loss = 0.164, train_accuracy = 0.725
epoch=2/393, val loss = 0.232, val_accuracy = 0.671
epoch: 3
epoch=3/393, train loss = 0.150, train_accuracy = 0.760
epoch=3/393, val loss = 0.223, val_accuracy = 0.649
epoch: 4
epoch=4/393, train loss = 0.130, train_accuracy = 0.800
epoch=4/393, val loss = 0.202, val_accuracy = 0.692
epoch: 5
epoch=5/393, train loss = 0.102, train_accuracy = 0.880
epoch=5/393, val loss = 0.193, val_accuracy = 0.700
epoch: 6
epoch=6/393, train loss = 0.082, train_accuracy = 0.885
epoch=6/393, val loss = 0.175, val_accuracy = 0.724
epoch: 7
epoch=7/393, train loss = 0.072, train_accuracy = 0.875
epoch=7/393, val loss = 0.177, val_accuracy = 0.712
epoch: 8
epoch=8/393, train loss = 0.097, train_accuracy = 0.830

Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0


32th patient: /cis/home/jhu104/jhu101/OTS-23-17323 - 2023-06-16 13.26.15.ndpi
0th test accuracy:1.000
test_accuracy = 0.920
Classification Results
viable: classification result for stroma: 0.00%
viable: classification result for viable: 84.00%
viable: classification result for necrosis: 16.00%
viable: classification result for cartilage: 0.00%
necrosis: classification result for stroma: 0.00%
necrosis: classification result for viable: 0.00%
necrosis: classification result for necrosis: 100.00%
necrosis: classification result for cartilage: 0.00%

11th patient: /cis/home/jhu104/jhu101/OTS-23-17323-031.ndpi
0th test accuracy:1.000
test_accuracy = 0.870
Classification Results
viable: classification result for stroma: 6.00%
viable: classification result for viable: 86.00%
viable: classification result for necrosis: 8.00%
viable: classification result for cartilage: 0.00%
necrosis: classification result for stroma: 12.00%
necrosis: classification result for viable: 0.00%
necrosis: classifi

In [31]:
#step0
#Read OSTU dictoinary
with open("/cis/home/jhu104/osteosarcoma/OSTU/ostu.dic", 'r') as j:
    adjust_otsu_all1 = json.loads(j.read())

#step1
#Get basename dictionray
data_dic = {}
for key,value in adjust_otsu_all1.items():
    basename = os.path.basename(key)
    data_dic[basename] = value

fold_index = [3,4,5,89]
accuracy = {}
for fold in range(len(fold_index)):
    print(f"!!!!!!!!!!!!Fold:{fold_index[fold]}")
    #step2
    #Read training file from zhenzhen
    split_df = pd.read_csv(f"/cis/home/jhu104/osteosarcoma/OSTU/osteosarcoma_experiment_setting{fold_index[fold]}.csv")


    #step3
    #change the dictionary to df
    ostu_df = pd.DataFrame(data_dic.items())
    ostu_df.rename(columns={0: "image_name", 1: "OSTU"}, inplace=True)


    #step4
    #merge the table
    split_df=split_df.merge(ostu_df,on = ["image_name"])
    split_df=split_df.sort_values(by="dataset")



    #Get all the label
    label_list = []
    directory = '/export/io86/data/jhu101'
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            label_list.append(filepath)


    #Find matched label
    def find_matched_label(image,label):
        label_match = []
        for i in range(len(image)):
            appendix = os.path.basename(image[i])
            appendix = appendix+".xml"
            for j in range(len(label)):
                if appendix == os.path.basename(label_list[j]):
                    label_match.append(label_list[j])
        return label_match


    #Find all image path
    random.seed(42)
    directory = '/cis/home/jhu104/jhu101'
    datalist=[]
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            datalist.append(filepath)
    #randomly shuffle the dataset
    random.shuffle(datalist)


    #Get train,test and validation data path
    def get_data(directory):
        base = '/cis/home/jhu104/jhu101'
        datatrain=[]
        for i in range(len(directory)):
            filepath = os.path.join(base, directory[i])
            datatrain.append(filepath)
        return datatrain


    #Split into training,validation and test
    #train
    train_data = get_data(split_df[split_df["dataset"] =="training"]["image_name"].to_list())
    train_label = find_matched_label(train_data,label_list)

    #val
    val_data = get_data(split_df[split_df["dataset"] =="validation"]["image_name"].to_list())
    val_label = find_matched_label(val_data,label_list)

    #test
    test_data = get_data(split_df[split_df["dataset"] =="test"]["image_name"].to_list())
    test_label = find_matched_label(test_data,label_list)

    #Find settings for slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 10)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/2 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic

    #Get the bag length
    Settings = slide_setting(train_data,train_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides

    #Get the train setting

    adjust_otsu_train = split_df[split_df["dataset"] =="training"]["OSTU"].to_list()
    Settings =insert_otsu(Settings,adjust_otsu_train)

    #Load all training WSIs
    unit = 256
    level = 2 # 10x magnification
    patch_shape = 256
    annotation_label_mapping ={
        'stroma':1,
        'viable':2,
        'necrosis':3,
        'type4':4
    }
    slides_train = {}
    num_bags_train = {}
    annotations_train = {}
    tissue_masks_train = {}
    label_masks_train = {}
    for i, slide_ID in enumerate(train_data):
        slides_train[slide_ID] = openslide.OpenSlide(slide_ID)
        annotations_train[slide_ID] = read_Aaron_annotations(train_label[i])
        num_bags_train[slide_ID] = [Settings[slide_ID]['stroma'],Settings[slide_ID]['viable'],Settings[slide_ID]['necrosis'],Settings[slide_ID]['type4']]    
        tissue_masks_train[slide_ID] = generate_tissue_mask(slides_train[slide_ID],unit=unit,adjust_otsu=Settings[slide_ID]['adjust_otsu'])
        label_masks_train[slide_ID] = generate_label_mask(slides_train[slide_ID], annotations_train[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length
    train_bag_len =[0,0,0,0]
    for idx,key in num_bags_train.items():
        train_bag_len[0] += key[0] 
        train_bag_len[1] += key[1] 
        train_bag_len[2] += key[2] 
        train_bag_len[3] += key[3]

    #Find settings for validation slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 20)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/8 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic


    #Get the bag length
    Settings_val = slide_setting(val_data,val_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides


    adjust_otsu_val= split_df[split_df["dataset"] =="validation"]["OSTU"].to_list()
    Settings_val =insert_otsu(Settings_val,adjust_otsu_val)   


    #Load validation WSI
    if len(val_data)>0:
        slides_val = {}
        num_bags_val = {}
        annotations_val= {}
        tissue_masks_val = {}
        label_masks_val = {}
        for i, slide_ID in enumerate(val_data):
            slides_val[slide_ID] = openslide.OpenSlide(slide_ID)
            annotations_val[slide_ID] = read_Aaron_annotations(val_label[i])
            num_bags_val[slide_ID] = [Settings_val[slide_ID]['stroma'],Settings_val[slide_ID]['viable'],Settings_val[slide_ID]['necrosis'],Settings_val[slide_ID]['type4']]    
            tissue_masks_val[slide_ID] = generate_tissue_mask(slides_val[slide_ID],unit=unit,adjust_otsu=Settings_val[slide_ID]['adjust_otsu'])
            label_masks_val[slide_ID] = generate_label_mask(slides_val[slide_ID], annotations_val[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length

    val_bag_len =[0,0,0,0]
    for idx,key in num_bags_val.items():
        val_bag_len[0] += key[0] 
        val_bag_len[1] += key[1]
        val_bag_len[2] += key[2] 
        val_bag_len[3] += key[3] 

    #Find settings for test slides
    def slide_setting(data,label):
        dic = {}
        for i in range(len(data)):
            Annotations=read_Aaron_annotations(label[i])
            slide = openslide.OpenSlide(data[i])
            Mask = generate_label_mask(slide,Annotations,annotation_label_mapping,256)
            # print(f"""{i+1}st slide: viable: {round(np.sum(np.array(Mask)==1)/20)},
            #                         necrosis: {round(np.sum(np.array(Mask)==2)/20)}, 
            #                         stroma: {round(np.sum(np.array(Mask)==3)/20)}, type4: {round(np.sum(np.array(Mask)==4)/20)}""")

            # viable = round(np.sum(np.array(Mask) == 1) / 20)
            # necrosis = round(np.sum(np.array(Mask) == 2) / 20)
            # stroma = round(np.sum(np.array(Mask) == 3) / 20)
            # type4 = round(np.sum(np.array(Mask) == 4)/8 ) 
            viable = 500
            necrosis = 500
            stroma = 500
            type4 = 500
            dic[(data[i])] ={
                'stroma': stroma,
                'viable': viable,
                'necrosis': necrosis,
                'type4': type4,
            }
        return dic


    #Get the bag length
    Settings_test = slide_setting(test_data,test_label)

    #Insert the OTSU
    def insert_otsu(settingslides,adjust_otsu_type):
        for key, adjust_otsu in zip(settingslides.keys(), adjust_otsu_type):
            settingslides[key]['adjust_otsu'] = adjust_otsu
        return settingslides


    adjust_otsu_test= split_df[split_df["dataset"] =="test"]["OSTU"].to_list()
    Settings_test =insert_otsu(Settings_test,adjust_otsu_test)

    #Load test WSI
    if len(test_data)>0:
        slides_test= {}
        num_bags_test = {}
        annotations_test= {}
        tissue_masks_test = {}
        label_masks_test = {}
        for i, slide_ID in enumerate(test_data):
            slides_test[slide_ID] = openslide.OpenSlide(slide_ID)
            annotations_test[slide_ID] = read_Aaron_annotations(test_label[i])
            num_bags_test[slide_ID] = [Settings_test[slide_ID]['stroma'],Settings_test[slide_ID]['viable'],Settings_test[slide_ID]['necrosis'],Settings_test[slide_ID]['type4']]    
            tissue_masks_test[slide_ID] = generate_tissue_mask(slides_test[slide_ID],unit=unit,adjust_otsu=Settings_test[slide_ID]['adjust_otsu'])
            label_masks_test[slide_ID] = generate_label_mask(slides_test[slide_ID], annotations_test[slide_ID], annotation_label_mapping, unit)

    #Find the train bag total length
    test_bag_len =[0,0,0,0]
    for idx,key in num_bags_test.items():
        test_bag_len[0] += key[0] 
        test_bag_len[1] += key[1] 
        test_bag_len[2] += key[2]  
        test_bag_len[3] += key[3] 

    wandb.init(
        project= "vgg16",
        name = "vgg -scratch",
        config={
        "learning_rate": 1e-5,
        "architecture": "vgg16_lower_lr"
        }
    )
    random.seed(20)
    np.random.seed(20)
    torch.manual_seed(20)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(20)
    config = wandb.config
    device = torch.device("cuda:1")
    torch.cuda.set_device(device)
    model = Attention_modern_multi(load_vgg16_tune(),True)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(),lr=1e-6, betas=(0.9, 0.999), weight_decay =10e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma = 0.95)
    train_bag_len = [600,600,600,600]
    val_bag_len = [50,50,50,50] 
    Dataset_train = create_dataset_mixpatch(slides_train,
                                    tissue_masks_train, label_masks_train, 
                                    train_bag_len, level, patch_shape,length_bag_mean = 10)
    if len(slides_val)>0:
        Dataset_val = create_dataset_mixpatch(slides_val,
                                    tissue_masks_val, label_masks_val, 
                                    val_bag_len, level, patch_shape,length_bag_mean = 10)
        model, Train_loss, Train_accuracy, Val_loss, Val_accuracy = Train(model, Dataset_train, optimizer, scheduler, fold_index[fold],validation=True, Dataset_val = Dataset_val)
        
    else:
        model, Train_loss, Train_accuracy, Val_loss, Val_accuracy = Train(model, Dataset_train, optimizer, scheduler,fold_index[fold], validation=False, Dataset_val = None)

    #step0
    # load model
    #Find patient ID for each slide
    td   = split_df[split_df["dataset"]=="test"]
    test_data_dic = {}
    for index,row in td.iterrows():
        image_name = "/cis/home/jhu104/jhu101/"+ row["image_name"]
        patient_id = row["patient id"]
        test_data_dic[image_name] = patient_id
    # f = open("/cis/home/jhu104/osteosarcoma/validation and test result/test_data_dic.json")
    # test_data_dic = json.load(f)
    path = f"/cis/home/jhu104/osteosarcoma/cross_validation_fold1/best_model_{fold_index[fold]}.pth"
    # device = torch.device("cuda:0")
    model = Attention_modern_multi(load_vgg16_tune())
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    set_bag_length =50
    test_bag_len_slide = [set_bag_length,set_bag_length,set_bag_length,set_bag_length]


        # 'stroma':1,
        # 'viable':2,
        # 'necrosis':3,
        # 'type4':4


    #step1
    #Prediction accuracy    
    def prediction_accuracy(pred,true,test_bag_len_slide):
        listname = ['stroma', 'viable', 'necrosis', 'cartilage']
        label_type = int(len(pred)/test_bag_len_slide[0])
        label_name = np.unique(true)
        #Get classification result
        for i in range(label_type):
            for j in range(len(listname)):
                class_len = int(len(pred)/label_type)
                result = np.sum(np.array(pred[i*class_len:(i+1)*class_len]) ==j)/class_len
                print(f"{listname[label_name[i]]}: classification result for {listname[j]}: {result*100:.2f}%")



    #step2
    #Prediction patient accuracy
    def prediction_patient_accuracy(pred,true,test_bag_len_slide):
        listname = ['stroma', 'viable', 'necrosis', 'cartilage']
        label_type = int(len(pred)/test_bag_len_slide[0])
        label_name = np.unique(true)
        stroma_acc = []
        viable_acc= []
        necrosis_acc = []
        cartilage_acc =[]
        #Get classification result
        for i,type in enumerate(label_name):
            # print(f"prediction type:{pred}")
            class_len = int(len(pred)/label_type)
            result = (np.sum(np.array(pred[i*class_len:(i+1)*class_len]) ==type)/class_len)
            if type == 0:
                stroma_acc.append(result)
            elif type ==1:
                viable_acc.append(result)
            elif type ==2:
                necrosis_acc.append(result)
            elif type ==3:
                cartilage_acc.append(result)
        return   stroma_acc,viable_acc,necrosis_acc,cartilage_acc




    #step3
    #Compute test accuracy
    # dict(itertools.islice(test_data_dic.items(), 2,16))
    # filtered_dict = {key: value for key, value in test_data_dic.items() if value == 33}
    patient_list = td["patient id"].unique().tolist()
    patient_metrics = {patient_id: {'stroma': [], 'viable': [], 'necrosis': [], 'cartilage': []} for patient_id in patient_list}
    test_accuracy_list = {} #####Slide level accuracy

    for keys,patient_id in test_data_dic.items():
        Dataset_test = create_dataset_mixbag(slides_test[keys],
                                        tissue_masks_test[keys], label_masks_test[keys], 
                                        test_bag_len_slide, level, patch_shape,length_bag_mean = 10)

        print(f"{patient_id}th patient: {keys}")
        #Test accuracy
        Dataloader_test = data_utils.DataLoader(Dataset_test, batch_size = 1, shuffle = False)
        pred_labels_test, true_labels,test_accuracy = test(model, Dataloader_test)
        basename = os.path.basename(keys)
        test_accuracy_list[basename]=round(test_accuracy,3)
        
        #Print slide accuracy
        print("test_accuracy = {:.3f}".format(test_accuracy))
        print("Classification Results")
        prediction_accuracy(pred_labels_test,true_labels,test_bag_len_slide)
        stroma_acc,viable_acc,necrosis_acc,cartilage_acc=prediction_patient_accuracy(pred_labels_test,true_labels,test_bag_len_slide)
        
    # Append the results for this key to the patient's metrics
        if stroma_acc:  
            patient_metrics[patient_id]['stroma'].append(stroma_acc[0])
        if viable_acc:  # 
            patient_metrics[patient_id]['viable'].append(viable_acc[0])
        if necrosis_acc: 
            patient_metrics[patient_id]['necrosis'].append(necrosis_acc[0])
        if cartilage_acc:  #
            patient_metrics[patient_id]['cartilage'].append(cartilage_acc[0])
        print("")



    #Step4
    #Compute patient level accuracy
    print("Patient Level Accuracy")
    for patient_id, metrics in patient_metrics.items():
        print("")
        for metric ,values in metrics.items():
            print(f"{patient_id}th: {metric}%: {np.mean(np.array(values))*100:.2f}(std:{np.std(np.array(values))*100:.2f})")

    #step5
    #Mean Acc and Std
    print("")
    print(f"Mean Accuracy:{np.mean(np.array(list(test_accuracy_list.values())))*100:.2f}(std:{np.std(np.array(list(test_accuracy_list.values())))*100:.2f})")
    with open("/cis/home/jhu104/osteosarcoma/validation and test result/slide_accuracy", "w") as f:
        # Use json.dump to write the dictionary to the file with indentation
        json.dump(test_accuracy_list, f, indent=4)  

    test_accuracy_list_updated={key:round(value*100,3) for key, value in test_accuracy_list.items()}
    # save model
    accuracy[fold_index[fold]] = test_accuracy_list_updated
    test_accuracy_list_updated
    with open("/cis/home/jhu104/osteosarcoma/validation and test result/test_accuracy_list_updated_last.json","w") as f:
        json.dump(accuracy,f,indent=4)

!!!!!!!!!!!!Fold:3


learning_rate,▁
train_accuracy,▁
train_loss,▁
learning_rate,0.0
train_accuracy,0.495
train_loss,0.4501


Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch: 0
epoch=0/357, train loss = 0.450, train_accuracy = 0.495
epoch=0/357, val loss = 0.327, val_accuracy = 0.524
epoch: 1
epoch=1/357, train loss = 0.208, train_accuracy = 0.705
epoch=1/357, val loss = 0.291, val_accuracy = 0.550
epoch: 2
epoch=2/357, train loss = 0.174, train_accuracy = 0.725
epoch=2/357, val loss = 0.253, val_accuracy = 0.591
epoch: 3
epoch=3/357, train loss = 0.122, train_accuracy = 0.800
epoch=3/357, val loss = 0.280, val_accuracy = 0.593
epoch: 4
epoch=4/357, train loss = 0.108, train_accuracy = 0.785
epoch=4/357, val loss = 0.293, val_accuracy = 0.607
epoch: 5
epoch=5/357, train loss = 0.114, train_accuracy = 0.785
epoch=5/357, val loss = 0.246, val_accuracy = 0.622
epoch: 6
epoch=6/357, train loss = 0.109, train_accuracy = 0.815
epoch=6/357, val loss = 0.257, val_accuracy = 0.632
epoch: 7
epoch=7/357, train loss = 0.100, train_accuracy = 0.820
epoch=7/357, val loss = 0.260, val_accuracy = 0.659
epoch: 8
epoch=8/357, train loss = 0.079, train_accuracy = 0.820

Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0


...

Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch: 0
epoch=0/456, train loss = 0.427, train_accuracy = 0.435
epoch=0/456, val loss = 0.371, val_accuracy = 0.473
epoch: 1
epoch=1/456, train loss = 0.269, train_accuracy = 0.555
epoch=1/456, val loss = 0.291, val_accuracy = 0.570
epoch: 2
epoch=2/456, train loss = 0.203, train_accuracy = 0.710
epoch=2/456, val loss = 0.236, val_accuracy = 0.623
epoch: 3
epoch=3/456, train loss = 0.141, train_accuracy = 0.760
epoch=3/456, val loss = 0.196, val_accuracy = 0.648
epoch: 4
epoch=4/456, train loss = 0.120, train_accuracy = 0.815
epoch=4/456, val loss = 0.173, val_accuracy = 0.692
epoch: 5
epoch=5/456, train loss = 0.134, train_accuracy = 0.795
epoch=5/456, val loss = 0.171, val_accuracy = 0.713
epoch: 6
epoch=6/456, train loss = 0.119, train_accuracy = 0.820
epoch=6/456, val loss = 0.161, val_accuracy = 0.746
epoch: 7
epoch=7/456, train loss = 0.088, train_accuracy = 0.875
epoch=7/456, val loss = 0.154, val_accuracy = 0.743
epoch: 8
epoch=8/456, train loss = 0.076, train_accuracy = 0.895

Using cache found in /cis/home/jhu104/.cache/torch/hub/pytorch_vision_v0.6.0


35th patient: /cis/home/jhu104/jhu101/OTS-23-17323 - 2023-06-16 15.53.59.ndpi
0th test accuracy:1.000
test_accuracy = 0.700
Classification Results
stroma: classification result for stroma: 100.00%
stroma: classification result for viable: 0.00%
stroma: classification result for necrosis: 0.00%
stroma: classification result for cartilage: 0.00%
viable: classification result for stroma: 6.00%
viable: classification result for viable: 80.00%
viable: classification result for necrosis: 14.00%
viable: classification result for cartilage: 0.00%
necrosis: classification result for stroma: 70.00%
necrosis: classification result for viable: 0.00%
necrosis: classification result for necrosis: 30.00%
necrosis: classification result for cartilage: 0.00%

35th patient: /cis/home/jhu104/jhu101/OTS-23-17323 - 2023-06-16 15.43.44.ndpi
0th test accuracy:1.000
test_accuracy = 0.640
Classification Results
stroma: classification result for stroma: 100.00%
stroma: classification result for viable: 0.00%
st

/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/cis/home/jhu104/osteosarcoma/venv/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
